In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, Sequential, datasets, optimizers, losses

#'''
def preprocess(x, y):
    x = 2*tf.cast(x, dtype=tf.float32)/255. - 1
    y = tf.cast(y, dtype=tf.float32)
    return x, y
#'''

#下载cifar10数据集
(x_in, y_in), (x_test, y_test) = datasets.cifar10.load_data()

# 构建训练集、测试集对象，随即打乱，预处理，批量化
''' # 用以下语句进行预处理，效果与用map(preprocess)函数是一样的
x_in = tf.convert_to_tensor(x_in, dtype=tf.float32)/255.
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)/255.
y_in = tf.convert_to_tensor(y_in, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)
train_db = tf.data.Dataset.from_tensor_slices((x_in, y_in))
train_db = train_db.shuffle(1000).batch(128)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.batch(128)
'''

# 打印训练集的形状
print(x_in.shape, y_in.shape, x_test.shape, y_test.shape)
# 交换维度
y_in = tf.squeeze(y_in, axis=1)
y_test = tf.squeeze(y_test, axis=1)

# 打印训练集的形状
print(x_in.shape, y_in.shape, x_test.shape, y_test.shape)

#'''
train_db = tf.data.Dataset.from_tensor_slices((x_in, y_in))
train_db = train_db.shuffle(1000).map(preprocess).batch(128) # 通过map(preprocess)进行数据预处理。
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(64)
#'''


# 从训练集中采样一个batch，并观察
sample = next(iter(train_db))
print('sample', sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))


# 创建卷基层，5个
conv_layers = [ #创建包含多层网络的列表
    # 64个3✖️3卷积核，输入输出同大小
    # padding='VALID'时不会对图片做填充；
    # 64个核是随机生成的
    layers.Conv2D(64, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    # 高宽减半
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'), 

    # 128个3✖️3卷积核，输入输出同大小
    layers.Conv2D(128, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'), 
    
    # 256个3✖️3卷积核，输入输出同大小
    layers.Conv2D(256, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'), 
    
    # 512个3✖️3卷积核，输入输出同大小
    layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'), 
    
    # 512个3✖️3卷积核，输入输出同大小
    layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3,3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same')
]
conv_net = Sequential(conv_layers)

# 全联接层
fc_net = Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(10, activation=None),
])

# build 两个字网络，并打印参数信息
# build 的作用是在训练“fit”之前如果想查看网络结构，可以先用build建立起来网络
# build建立的网络不会带有权重，fit建立的是包括了训练的权重的网络
conv_net.build(input_shape=[None, 32, 32, 3])
fc_net.build(input_shape=[None, 512])
conv_net.summary()
fc_net.summary()



(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
sample (128, 32, 32, 3) (128,) tf.Tensor(-1.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1792      
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  73856     
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  147584    
__________________________

In [23]:
# 优化模型
# 设置损失函数、优化器
optimizer = optimizers.Adam(lr=0.001)
# criteon = keras.losses.CategoricalCrossentropy(from_logits=True) 
# 合并两个子网络参数，需要在model.build之后才能计算
variables = conv_net.trainable_variables + fc_net.trainable_variables



In [ ]:
for epoch in range(50):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [b, 32, 32, 3] ==> [b, 1, 1, 512]
            out = conv_net(x)
            print(out.shape)
            # 打平
            out = tf.reshape(out, [-1, 512])
            logits = fc_net(out)
#            y_onehot = tf.one_hot(y, depth=10)
            # calculate losses
#            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
#            loss = tf.reduce_mean(loss)
        # 计算梯度
#        grads = tape.gradient(loss, variables)
        # 更新权重
#        optimizer.apply_gradients(zip(grads, variables))
#        print(step, loss)

